First all relevant modules are imported

In [1]:
import numpy as np
import networkx as nx
from collections import defaultdict
import random
import pickle
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import collections
import itertools
import scipy as sp
import scipy.stats

import os

import osmnx as ox

from toysimulations import ZeroDetourBus, Stop, Request, Network
import route

ModuleNotFoundError: No module named 'osmnx'

The functions for creating the requests and running the simulation are defined.


In [2]:
def req_generator_uniform(graph, num_reqs, req_rate):
    """
    Generates requests with rate=req_rate whose origin and
    destination are drawn uniformly randomly. The requests
    are generated in time as a Poisson process.
    """
    t = 0
    req_idx = 0

    while req_idx < num_reqs:
        orig, dest = random.sample(graph.nodes(), k=2)
        delta_t = np.random.exponential(1/req_rate)

        t += delta_t
        req_idx += 1
        yield Request(req_idx, t, orig, dest)
        
def simulate_single_request_rate(G, nG, x,network_type, l_avg):
    """
    Simulates only as single request rate x. See the docstring of
    `simulate_different_request_rates` for details on the arguments.
    Large number of requests is chosen to create a long tour for better data reliability for the edge usages.
    """
    num_reqs = 100000
    req_rate = x/2/l_avg

    sim = ZeroDetourBus(nG,
                        req_generator_uniform(G, num_reqs, req_rate),
                        network_type, 
                        random.sample(G.nodes(), k=1)[0],
                       )
    print(f"simulating x={x}")
    sim.simulate_all_requests()
    return sim.req_data, sim.insertion_data

The functions for obtaining the link usage are defined as in the bachelor thesis of Karolin Stiller.

In [3]:

def obtain_tour_from_simulation(req_data):
    """


    Gets a dictionary req_data (generated by the simulator) containing for each request (indexed beginning with 1):
        req_epoch, origin, destination, pickup_epoch, dropoff-epoch
    Returns a list of the visited nodes in the right order each entry 
        (req_indx, point in grid, epoch time stamp) 
            where req_index is negative if the point in grid = origin of request 
                           and positive if the point in grid = destination of request
    
    """
    tour = []
    for request in req_data:
        tour.append([-request, req_data[request]['origin'], req_data[request]['pickup_epoch']])
        tour.append([request, req_data[request]['destination'], req_data[request]['dropoff_epoch']])

    tour.sort(key= lambda r: r[2])

    cutted_tour = [tour[0]]
    cutted_tour[0][0] = [cutted_tour[0][0]]

    tour_length = len(tour)
    for i in range(1, tour_length):
        if tour[i][1] == cutted_tour[-1][1]:
            cutted_tour[-1][0].append(tour[i][0])
        else:
            cutted_tour.append(tour[i])
            cutted_tour[-1][0] = [cutted_tour[-1][0]]

    return(cutted_tour)

def obtain_effective_topology_of_tour(G, nG, tour):
    '''
    Gets the graph G, its network, and the tour (in the sense of obtain_tour_from_simulation() )
    Returns a dictionary of the edges, specifying a scaled version of the number of times each edge is probably used in the tour
    If there are multiple shortest paths from one node to another - each of them could be chosen with same probability
    '''
    times_edge_used = [0 for e in G.edges]
    used_edges = dict(zip(G.edges, times_edge_used))
    tour_length = len(tour) 
    
    driven_distance = 0
    
    #This guarantees that the function works also if only the node sequence is passed on
    if type(tour[0]) == list:
        tour_with_stops_and_times = True
    else:
        tour_with_stops_and_times = False
    
    if nG.network_type == 'grid':
        #Das geht garantiert effizienter!!!!!
        for i in range(tour_length-1):
            if tour_with_stops_and_times == True:
                source = tour[i][1]
                target = tour[i+1][1]
            else:
                source = tour[i]
                target = tour[i+1]

            s_x, s_y = source
            t_x, t_y = target
            width = abs(t_x - s_x)
            hight = abs(t_y - s_y)
            length_shortest_path = width + hight
            number_of_shortest_paths = int(scipy.special.binom(width + hight, width))

            driven_distance += length_shortest_path
            
            paths = nx.all_shortest_paths(G, source, target)
            for k in range(number_of_shortest_paths):
                one_shortest_path = next(paths)

                for j in range(length_shortest_path):
                    
                    if (one_shortest_path[j], one_shortest_path[j+1]) in used_edges:
                        used_edges[(one_shortest_path[j], one_shortest_path[j+1])] += 1/number_of_shortest_paths
                    else:
                        used_edges[(one_shortest_path[j+1], one_shortest_path[j])] += 1/number_of_shortest_paths

    else:
        for i in range(tour_length-1):
            if tour_with_stops_and_times == True:
                source = tour[i][1]
                target = tour[i+1][1]
            else:
                source = tour[i]
                target = tour[i+1]

            length_shortest_path = nx.shortest_path_length(G, source=source, target=target)
            number_of_shortest_paths = sum(1 for p in nx.all_shortest_paths(G, source, target))

            driven_distance += length_shortest_path
            
            paths = nx.all_shortest_paths(G, source, target)
            for one_shortest_path in paths:
                for j in range(length_shortest_path):                    
                    if (one_shortest_path[j], one_shortest_path[j+1]) in used_edges:
                        used_edges[(one_shortest_path[j], one_shortest_path[j+1])] += 1/number_of_shortest_paths
                    else:
                        used_edges[(one_shortest_path[j+1], one_shortest_path[j])] += 1/number_of_shortest_paths

    #Scaling of the weights: Normalized to a totalweight of the original number of edges in G
    num = G.number_of_edges()
    #newtotal = 0 #Zur Pruefung, dass Gesamtgewicht richtig ist
    for edge in used_edges:
        used_edges[edge] = (used_edges[edge]/driven_distance) * num
        #newtotal += used_edges[edge]
    
    #print("Driven distance is %s" %driven_distance)
    #print("Newtotal is %s" %newtotal)
    return used_edges


On the example of Ingolstadt it is shown how the functions would be applied to a network.

In [ ]:
print("doing Ingolstadt")
graph_path = 'graph_ingolstadt.gpkl'
G = nx.read_gpickle(graph_path)
nG=Network(G,network_type='own')
l_avg = nx.average_shortest_path_length(G)
x=10
req_data,insertion_data=simulate_single_request_rate(G, nG, x, network_type='novolcomp', l_avg=l_avg)

cutted_tour=obtain_tour_from_simulation(req_data)
used_edges=obtain_effective_topology_of_tour(G, nG, cutted_tour)